# AI4Code

## Libraries

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
import math
import json
import warnings
from tqdm.notebook import tqdm; tqdm.pandas();
from glob import glob
import re
import os


import matplotlib.pyplot as plt

## Load data

In [2]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
paths = []
n= len('/kaggle/input/AI4Code/') 
for dirname, _, filenames in os.walk('/kaggle/input/AI4Code/'):
    
    #I don't want save train/test folders
    if len(dirname) == n: 
        for filename in filenames:
            print(os.path.join(dirname, filename))
            paths.append(os.path.join(dirname, filename))
    else:
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/AI4Code/train_orders.csv
/kaggle/input/AI4Code/sample_submission.csv
/kaggle/input/AI4Code/train_ancestors.csv


In [3]:
tOrd = pd.read_csv(paths[0])
tAnt = pd.read_csv(paths[2])

In [4]:
def splitB(string=" ", spl = " "):
    """
    For split by an element, blank space by default
    Example: splitB("Hello World!" , splt= " ") -> ["Hello", "World!"]
    """
    return string.split(spl)

### Train orders

Explode id orders

In [5]:
tOrd["cell_order"] = tOrd["cell_order"].apply(splitB)
tOrd = tOrd.explode("cell_order", ignore_index = True)

Enumerate id orders and set the size

In [6]:
sizes = tOrd.groupby('id', sort=False).size().values 
tOrd['enumCol'] = np.arange(sizes.sum()) - np.repeat(sizes.cumsum() - sizes, sizes)+1

In [7]:
sizesDF = pd.DataFrame(tOrd.groupby("id").size(), columns = ["sizeId"])
tOrd = tOrd.set_index("id").join( sizesDF , on = "id", how = "left", rsuffix = "Id").reset_index()

In [8]:
tOrd.head()

,id,cell_order,enumCol,sizeId
0,00001756c60be8,1862f0a6,1,58
1,00001756c60be8,448eb224,2,58
2,00001756c60be8,2a9e43d6,3,58
3,00001756c60be8,7e2f170a,4,58
4,00001756c60be8,038b763d,5,58


In [9]:
def loadJson(path):
    import pandas as pd
    tmpDf = pd.read_json(path, dtype={'cell_type': 'category', 'source': 'str'}).reset_index().rename({"index":"cell_order"}, axis=1)
    tmpDf["id"] = path.rsplit(".", 1)[0].rsplit("/", 1)[-1]
    return tmpDf

In [10]:
pathJson = glob(os.path.join("/kaggle/input/AI4Code/train", "*.json"), recursive=True)

In [11]:
#dfJsons = loadJson(pathJson[0])

#for path in tqdm(pathJson[1:]):
#    dfJsons = dfJsons.append(loadJson(path), ignore_index=True)